In [ ]:
#Installing the huggingface transformers library
!pip3 install transformers
!pip3 install datasets

     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 636 kB 54.1 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 3.3 MB 50.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 8.6 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 118 kB 71.6 MB/s 
     |████████████████████████████████| 243 kB 54.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
#Importing the necessary modules
from tensorflow import feature_column
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

#downloading Marbert pretrained model
model = TFAutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT")

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/652M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  162841344 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 162,842,882
Trainable params: 162,842,882
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from transformers import AutoTokenizer
#getting the MarBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("     ")

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#scikit learn splitting function:
from sklearn.model_selection import train_test_split

In [ ]:
#the waseet file for training
!gdown --id 1yj35VC8I1TBFuTYCq_nh2qddyIrD35JS

Downloading...
From: https://drive.google.com/uc?id=1yj35VC8I1TBFuTYCq_nh2qddyIrD35JS
To: /content/arabic-arabic_waseet_ghani_mu3aserh_raid.xlsx
17.8MB [00:00, 156MB/s]


In [ ]:
#splitting the training file into train and validation(95%: training, 5% for validation)
data = pd.read_excel('arabic-arabic_waseet_ghani_mu3aserh_raid.xlsx')
data = data[:10_000]
df_train, df_val = train_test_split(data, test_size=0.05, shuffle=True)
df_train.to_csv('train.csv')
df_val.to_csv('val.csv')
data.to_csv('data.csv')

In [ ]:
df_train.to_csv('train.csv')
df_val.to_csv('val.csv')


In [ ]:
#load the train and validatio data as tf datasets
from datasets import load_dataset
train_dataset = load_dataset('csv', data_files='train.csv')
val_dataset = load_dataset('csv', data_files='val.csv')


Using custom data configuration default-7c420ac120b05b91


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7c420ac120b05b91/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Using custom data configuration default-069c81747b5b52e3


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-069c81747b5b52e3/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [ ]:
#create our function for features tokenization 
def tokenize_function(examples):
    return tokenizer(examples["meaning"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)


  0%|          | 0/10 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
data_dataset = load_dataset('csv', data_files='data.csv')

#function for label tokenization:
def tokenize_labels(examples):
    return tokenizer(examples["word"], padding="max_length", truncation=True)
tokenized_labels_train = train_dataset.map(tokenize_labels, batched=True)
tokenized_labels_val = val_dataset.map(tokenize_labels, batched=True)


Using custom data configuration default-3ef64a5fdb0c629f


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3ef64a5fdb0c629f/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#converting the train and validation data to tf formats:
full_train_dataset = tokenized_train['train']
full_val_dataset = tokenized_val['train']
full_train_labels_dataset = tokenized_labels_train['train']
full_val_labels_dataset = tokenized_labels_val['train']

In [ ]:
tf_train_dataset = full_train_dataset.remove_columns('meaning').with_format("tensorflow")
tf_val_dataset = full_val_dataset.remove_columns('meaning').with_format("tensorflow")

tf_train_labels_dataset = full_train_labels_dataset.remove_columns('word').with_format('tensorflow')
tf_val_labels_dataset = full_val_labels_dataset.remove_columns('word').with_format('tensorflow')

In [ ]:
tf_train_labels_dataset

Dataset({
    features: ['Unnamed: 0', 'attention_mask', 'complement', 'input_ids', 'meaning', 'root', 'token_type_ids'],
    num_rows: 9500
})

In [ ]:
train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_labels_dataset['Unnamed: 0']))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)
#for the val data set
val_features = {x: tf_val_dataset[x].to_tensor() for x in tokenizer.model_input_names}
val_tf_dataset = tf.data.Dataset.from_tensor_slices((val_features, tf_val_labels_dataset['Unnamed: 0']))
val_tf_dataset = val_tf_dataset.batch(8)

/usr/local/lib/python3.7/dist-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)


In [ ]:
"""
    Compiling our model with:
"""
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=2e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.metrics.SparseCategoricalAccuracy(), ]
)

hist = model.fit(train_tf_dataset, batch_size=32, validation_data=val_tf_dataset, epochs=5, verbose=1)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr